In [ ]:
import os
import requests
import csv
from datetime import datetime, timezone
import time

# Create a session with a browser-like User-Agent
session = requests.Session()
session.headers.update({
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36"
})

def fetch_data_from_api(url):
    try:
        response = session.get(url, allow_redirects=True)
        print(f"Status Code: {response.status_code}")  # Debugging output
        
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Failed to fetch data. Status: {response.status_code}\nResponse Text: {response.text}")
            return None
    except Exception as e:
        print(f"Request failed: {e}")
        return None

def add_timestamp_to_data(data, timestamp):
    if data:
        for subarray_index, subarray in enumerate(data):
            for entry in subarray:
                entry['timestamp'] = timestamp
                entry['subarray'] = subarray_index
        return data
    else:
        return None

def save_to_csv(data, file_path):
    if data:
        with open(file_path, 'w', newline='') as csvfile:
            fieldnames = ['x', 'y', 'timestamp', 'subarray']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            for subarray in data:
                writer.writerows(subarray)

def check_integrity(data, file_path):
    with open(file_path, 'r') as csvfile:
        csv_reader = csv.DictReader(csvfile)
        csv_data = list(csv_reader)
        original_row_count = sum(len(subarray) for subarray in data)
        csv_row_count = len(csv_data)
        if original_row_count == csv_row_count:
            print(f"CSV file '{file_path}' integrity check passed.")
        else:
            print(f"CSV file '{file_path}' integrity check failed.")

api_url = 'https://energie.theoxygent.nl/api/prices_v2.php'

# Create subfolder if it doesn't exist
subfolder = 'oxygent_data'
if not os.path.exists(subfolder):
    os.makedirs(subfolder)

while True:
    current_time_gmt = datetime.now(timezone.utc).isoformat()
    file_timestamp = datetime.now(timezone.utc).strftime("%Y%m%d %H:%M")
    time_series_data = fetch_data_from_api(api_url) 
    time_series_data_with_timestamp = add_timestamp_to_data(time_series_data, current_time_gmt)
    csv_file_path = os.path.join(subfolder, f'time_series_data_{file_timestamp}.csv')
    save_to_csv(time_series_data_with_timestamp, csv_file_path)
    # Add a short delay to ensure the file is written before reading
    time.sleep(1)
    check_integrity(time_series_data_with_timestamp, csv_file_path)
    # Fetch data every 4 hour
    time.sleep(4*3600)

Status Code: 200
CSV file 'oxygent_data/time_series_data_20250404 07:01.csv' integrity check passed.


In [ ]:
''' This code stopped working 18 March 2024, giving the following error message: _Status Code: 403
Response Text: <html><body><h1>403 Forbidden</h1>
Request forbidden by administrative rules.
</body></html>

Failed to fetch data. Status: 403_''
''

''
import os
import requests
import csv
from datetime import datetime, timezone
import time

def fetch_data_from_api(url):
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print("Failed to fetch data")
        return None

def add_timestamp_to_data(data, timestamp):
    if data:
        for subarray_index, subarray in enumerate(data):
            for entry in subarray:
                entry['timestamp'] = timestamp
                entry['subarray'] = subarray_index
        return data
    else:
        return None

def save_to_csv(data, file_path):
    if data:
        with open(file_path, 'w', newline='') as csvfile:
            fieldnames = ['x', 'y', 'timestamp', 'subarray']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            for subarray in data:
                writer.writerows(subarray)

def check_integrity(data, file_path):
    with open(file_path, 'r') as csvfile:
        csv_reader = csv.DictReader(csvfile)
        csv_data = list(csv_reader)
        original_row_count = sum(len(subarray) for subarray in data)
        csv_row_count = len(csv_data)
        if original_row_count == csv_row_count:
            print(f"CSV file '{file_path}' integrity check passed.")
        else:
            print(f"CSV file '{file_path}' integrity check failed.")

api_url = 'https://energie.theoxygent.nl/api/prices_v2.php'

# Create subfolder if it doesn't exist
subfolder = 'oxygent_data'
if not os.path.exists(subfolder):
    os.makedirs(subfolder)

while True:
    current_time_gmt = datetime.now(timezone.utc).isoformat()
    file_timestamp = datetime.now(timezone.utc).strftime("%Y%m%d %H:%M")
    time_series_data = fetch_data_from_api(api_url) 
    time_series_data_with_timestamp = add_timestamp_to_data(time_series_data, current_time_gmt)
    csv_file_path = os.path.join(subfolder, f'time_series_data_{file_timestamp}.csv')
    save_to_csv(time_series_data_with_timestamp, csv_file_path)
    check_integrity(time_series_data_with_timestamp, csv_file_path)
    # Fetch data every hour
    time.sleep(3600)


Failed to fetch data


FileNotFoundError: [Errno 2] No such file or directory: 'oxygent_data/time_series_data_20250318 13:56.csv'